In [1]:
#read all files into an array (dataframes)
import numpy as np
import os
import pandas as pd
from IPython.display import display
import datetime
import re
import json
directory = os.getcwd() 

def filter_one_or_zero(n):
  return 1 if (n) else 0 

def split_wares(wares):
  if wares=="{}":
    return None
  res = re.sub(r".*wares\:", "", str(wares))
  return json.loads(res)

def split_inputqueus(inputqueu):
  if (len(inputqueu)==2):
    return dict({"none":0})

  try:
    res = json.loads(inputqueu)
    for i in res.keys():
      res[i] = json.loads(res[i])[2]
    return res
  except:
    return 'error:' + str(inputqueu) 
#  res = json.loads(inputqueu)
#  for i in res.keys():
#    res[i] = json.loads(res[i])[2]
#  return res
      

def read_data_from_files(directory):
  dataframes= []
  for root,dirs,files in os.walk(directory):
      for file in files:
         if (file.endswith("Buildingslogs.csv") & os.path.exists(file)):
            col_names = pd.read_csv(file, nrows=0).columns

            dataframes.append(pd.read_csv(file,sep=";",low_memory=False,header=[0],skiprows=[1]))
  return dataframes


#group the data by building where d[str(serial)] = group

def group_data_by_building(data):
  d = {}
  for data in data :
    for serial, group in data.groupby('serial'):
      d[str(serial)] = group
  return d


def int_to_time(input,hours_mapping):
  return datetime.datetime(2021, 1, 1) + datetime.timedelta(hours=input/hours_mapping)

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +datetime.timedelta(hours=t.minute//30))  

def map_to_real_time(data):
  # hou much is an hour in game time 
  # for example 1hour = 2000 game time unite
  res=data
  hour_in_numbers = (int(data[0]["time"].max()) - int(data[0]["time"].min())) / (365*24)
  for df in res :
    df["time"]=df["time"].apply(int_to_time,args=[hour_in_numbers])
    #df["time"]=df["time"].apply(hour_rounder)
    df.time=df.time.round("0.1S")
  return res  

def read_orders_data_from_files(directory):
  dataframes= []
  for root,dirs,files in os.walk(directory):
      for file in files:
         if (file.endswith("orderslogs.csv") & os.path.exists(file)):
            col_names = pd.read_csv(file, nrows=0).columns

            dataframes.append(pd.read_csv(file,sep=";",low_memory=False,header=[0],skiprows=[1]))
  return dataframes

#get one buildings stockes according to IBP Stock
#data_line is one line where wares != nan of the original data
def data_line_to_one_building_stock_IBP(position):
  #locations = pd.DataFrame(columns=['LOCNO','LOCTYPE','NAME1','NAME2','LAND1','REGIO','ORT01','PSTLZ','PSTL2','STRAS','LOEVM','TIME_ZONE','TELF1','FABKL','KUKLA','NODETYPE','CUST_ATTR1','CUST_ATTR2','CUST_ATTR3','CUST_ATTR4','CUST_ATTR5','CUST_ATTR6','CUST_ATTR7','CUST_ATTR8','CUST_ATTR9','CUST_ATTR10','LONGITUDE','LATITUDE'])
  #stocks = pd.DataFrame(columns=['MATNR','LOC_WERKS','VRFKZ','MNG01'])
  locatins=["0"]*4
  locatins[0]=position
  locatins[1]="V"
  locatins[2]="CET"
  locatins[3]="03"
  #locations["LOCNO"]=position
  #locations["LOCTYPE"]="V"
  #locations["TIME_ZONE"]="CET"
  #locations["FABKL"]="03"
  return locatins
  #group_data_by_building

def write_series_of_location_to_csv_as_IBPlocation(locationseries_nameseries,filename):

  IBPlocations = pd.DataFrame(columns=['LOCID','GEOLATITUDE','GEOLONGITUDE','LOCATIONDEL','LOCATIONREGION','LOCATIONTYPE','LOCCITY','LOCCOUNTRY','LOCNAME','LOCNAME2','LOCTIMEZONE','PHONE','PLANTTYPE','POBOXZIP','PRODUCTIONCALID','STREET','ZIP'])
  IBPlocations[["LOCID","LOCNAME"]]=locationseries_nameseries
  
  #x = set([x[0] for x in test["position"]])
  #locations["'LATITUDE'"]=list(x)
  #IBPlocations["MANDT"]="001"
  IBPlocations["LOCATIONDEL"]=" "
  #IBPlocations["LOGSYS"]="WDL"
  IBPlocations["LOCATIONTYPE"]="P"
  IBPlocations["LOCTIMEZONE"]="CET"
  IBPlocations["PRODUCTIONCALID"]="01"
  IBPlocations["PLANTTYPE"]="PL"
  #IBPlocations.drop_duplicates(subset ="LOCNO",keep = False, inplace = True)
  IBPlocations.to_csv("locations"+filename+ ".csv",sep=";",index=False,header=True)

def write_singel_string_wares_attribute_to_csv_as_IBPmaterial(wares):
  IBPmaterial = pd.DataFrame(columns=['MANDT','LOGSYS','MATNR','LVORM','MATKL','MEINS','PRDHA','MTART','CUST_ATTR1','CUST_ATTR2','CUST_ATTR3','CUST_ATTR4','CUST_ATTR5','CUST_ATTR6','CUST_ATTR7','CUST_ATTR8','CUST_ATTR9','CUST_ATTR10'])

  wares = re.sub(r".*wares\:", "", wares)
  Dict = json.loads(wares)
  wares_amount_data = pd.DataFrame.from_dict(Dict.items())

  wares_amount_data.columns = ['wares', 'amount']
  IBPmaterial["MATNR"]=wares_amount_data["wares"]
  IBPmaterial["MEINS"]="piece"
  IBPmaterial["MANDT"]="001"
  IBPmaterial["LOGSYS"]="WDL"
  IBpmaterialdes = pd.DataFrame(columns=["MANDT","LOGSYS","RESID","SPRAS","KTEXT"])
  IBpmaterialdes["RESID"]=IBPmaterial["MATNR"]
  IBpmaterialdes["SPRAS"]="E"
  IBpmaterialdes["MANDT"]="001"
  IBpmaterialdes["LOGSYS"]="WDL"
  IBpmaterialdes.to_csv("materialsdesc.csv",sep=";",index=False,header=False)
  IBPmaterial.to_csv("materials.csv",sep=";",index=False,header=False)


def write_product_location(serial,wares,input_queues,is_warehouse,is_production_site):
  if is_warehouse:
    IBPproduct_location = pd.DataFrame(columns=["LOCID","PRDID","ASSEMBLYSCRAP","DEMANDSUBSTTYPE","DETPLANNINGSCHEDULINGHORIZON",
    "FAIRSHAREEXCESSSTOCKSEGMENT","FAIRSHAREMAXSTOCKSEGMENT","FAIRSHARESAFETYSTOCKSEGMENT","FORECASTCONSPROFILE",
    "FREEZEHORIZONDISTRIBUTION","FREEZEHORIZONPRODUCTION","GOODSRECEIPTPROCTIME","INBOUNDCALID",
    "LOCATIONPRODUCTDEL","MRPCONTROLLER","PLANDELETIONHORIZON","PLANNEDDELIVERYTIME","PMAXLOTSIZE",
    "PMINLOTSIZE","PPINTEGRATIONMODE","PROCUREMENTTYPE","PRODUCTALLOCATIONPROCEDURE","PROUNDING",
    "SAFETYSTOCKMD","SHIPPINGCALID","SOURCEOFSUPPLYSELPROFILE","SUBNETWORKID"])
    wares=wares
    wares = re.sub(r".*wares\:", "", wares)
    Dict = json.loads(wares)
    wares_amount_data = pd.DataFrame.from_dict(Dict.items())
    wares_amount_data.columns = ['wares', 'amount']
    IBPproduct_location["PRDID"]=wares_amount_data["wares"]
    #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
    IBPproduct_location['LOCID']=str(serial)
    IBPproduct_location["LOCATIONPRODUCTDEL"]=" "
    #IBPproduct_location["VRFKZ"]="X"
    IBPproduct_location.to_csv("product_location.csv",sep=";",index=False,mode='a',header=not os.path.exists("product_location.csv"))
  if is_production_site:
    IBPproduct_location = pd.DataFrame(columns=["LOCID","PRDID","ASSEMBLYSCRAP","DEMANDSUBSTTYPE","DETPLANNINGSCHEDULINGHORIZON",
    "FAIRSHAREEXCESSSTOCKSEGMENT","FAIRSHAREMAXSTOCKSEGMENT","FAIRSHARESAFETYSTOCKSEGMENT","FORECASTCONSPROFILE",
    "FREEZEHORIZONDISTRIBUTION","FREEZEHORIZONPRODUCTION","GOODSRECEIPTPROCTIME","INBOUNDCALID",
    "LOCATIONPRODUCTDEL","MRPCONTROLLER","PLANDELETIONHORIZON","PLANNEDDELIVERYTIME","PMAXLOTSIZE",
    "PMINLOTSIZE","PPINTEGRATIONMODE","PROCUREMENTTYPE","PRODUCTALLOCATIONPROCEDURE","PROUNDING",
    "SAFETYSTOCKMD","SHIPPINGCALID","SOURCEOFSUPPLYSELPROFILE","SUBNETWORKID"])
    wares=input_queues
    Dict = json.loads(wares)
    wares_amount_data=pd.DataFrame(columns=['wares', 'amount'])
    wares_amount_data['wares'] = Dict.keys()
    
    IBPproduct_location["PRDID"]=wares_amount_data["wares"]
    #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
    IBPproduct_location['LOCID']=str(serial)
    IBPproduct_location["LOCATIONPRODUCTDEL"]=" "
    #IBPproduct_location["VRFKZ"]="X"
    IBPproduct_location.to_csv("product_location.csv",sep=";",index=False,mode='a',header=not os.path.exists("product_location.csv"))

def write_singel_string_wares_attribute_to_csv_as_IBPproduct(wares):
  IBPmaterial = pd.DataFrame(columns=['PRDID','PRODDESC','PRODGROUP','PRODTYPE','PRODUCTDEL','PRODUCTHIERARCHY','UOMID'])

  wares = re.sub(r".*wares\:", "", wares)
  Dict = json.loads(wares)
  wares_amount_data = pd.DataFrame.from_dict(Dict.items())

  wares_amount_data.columns = ['wares', 'amount']
  IBPmaterial["PRDID"]=wares_amount_data["wares"]
  IBPmaterial["UOMID"]="ST"
  IBPmaterial['PRODUCTDEL']=" "
  IBPmaterial["PRODDESC"]="no desc lang E"

  #IBpmaterialdes = pd.DataFrame(columns=["MANDT","LOGSYS","RESID","SPRAS","KTEXT"])
  #IBpmaterialdes["RESID"]=IBPmaterial["MATNR"]
  #IBpmaterialdes["SPRAS"]="E"
  #IBpmaterialdes["MANDT"]="001"
  #IBpmaterialdes["LOGSYS"]="WDL"
  #IBpmaterialdes.to_csv("materialsdesc.csv",sep=";",index=False,header=False)
  IBPmaterial.to_csv("PRODUCT.csv",sep=";",index=False,header=True)

def write_one_buildings_entry_to_IBPstock(building_entry,filename):
  IBPstocks = pd.DataFrame(columns=['MANDT','DELKZ','LOGSYS','MATNR','LOC_WERKS','LGORT','PLAAB','BLANR','LOC_LIFNR','LOC_KUNNR','CHARG','TST01','VRFKZ','MNG01'])
  wares=building_entry["wares"]
  wares = re.sub(r".*wares\:", "", wares)
  Dict = json.loads(wares)
  wares_amount_data = pd.DataFrame.from_dict(Dict.items())
  wares_amount_data.columns = ['wares', 'amount']
  IBPstocks["MATNR"]=wares_amount_data["wares"]
  IBPstocks["MNG01"]=wares_amount_data["amount"]
  IBPstocks['LOC_WERKS']=str(building_entry["serial"])
  IBPstocks["VRFKZ"]="X"
  IBPstocks['MANDT']="001"
  IBPstocks['LOGSYS']="WDL"
  IBPstocks.to_csv(filename+"stocks.csv",sep=";",index=False,header=False)

def write_one_buildings_orders_to_IBPorders(orders):
  IBPorders = pd.DataFrame(columns=['MANDT','LOGSYS','DELNR','DELPS','DELET','DELKZ','MATNR','LOC_WERKS',
  'CHARG','PLAAB','PLANR','LGORT','LOC_LIFNR','LOC_KUNNR','TST01','VRFKZ','MNG01','MNG02',
  'MNG03','TST02','FIX01','LOC_WERKS_FROM','VERID','DELVR','AUFVR','POSVR',
  'INFNR','EKORG','ESOKZ','MNG04','DEPLOYMENT','MOT_ID','EINVR','DPS_TST','PP_STAGE','REL_FIXED_STATUS'])
  IBPorders['MANDT']="001"
  IBPorders['LOGSYS']="WDL"
  IBPorders['DELNR']=orders.serial.apply(str)+orders.name.apply(str)+orders.time.apply(str)+orders.requested_Ware.apply(str)+orders.amount.apply(str)
  IBPorders.to_csv("IBPorders.csv")

  
def rewrite_wares_to_one_simgle_ware(wares,ware):
  return wares[ware]


def write_pds_header(serial,wares,input_queues,is_warehouse,is_production_site):
  if is_warehouse:
    data_to_be_written_to_csv = pd.DataFrame(columns=["LOCID","PRDID","PRODUCTIONVERSION","ASSEMBLYSCRAP"
    ,"BOMEXPLOSIONRULE","OUTPUTCOEFFICIENT","PDSDEL"
    ,"PDSPRIO","PDSVALIDFR","PDSVALIDTO","PMAXLOTSIZE"
    ,"PMINLOTSIZE","PRODUCTIONVERSIONDESCR","PROUNDING"])

    if (wares != "na"):
        wares = re.sub(r".*wares\:", "", wares)
        Dict = json.loads(wares)
        wares_amount_data = pd.DataFrame.from_dict(Dict.items())
        wares_amount_data.columns = ['wares', 'amount']
        data_to_be_written_to_csv["PRDID"]=wares_amount_data["wares"]
    else: return

    
    #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
    data_to_be_written_to_csv['LOCID']=str(serial)
    data_to_be_written_to_csv["PDSDEL"]=" "
    data_to_be_written_to_csv["PRODUCTIONVERSION"]="01"
    data_to_be_written_to_csv["PDSPRIO"]=1
    data_to_be_written_to_csv["PRODUCTIONVERSIONDESCR"]="come on"
    data_to_be_written_to_csv["PDSVALIDFR"]="20160829,172223,X"
    data_to_be_written_to_csv["PDSVALIDTO"]="20240829,172223,X"
    data_to_be_written_to_csv["OUTPUTCOEFFICIENT"]=1
    #IBPproduct_location["VRFKZ"]="X"
    data_to_be_written_to_csv.to_csv("pds_header.csv",sep=";",index=False,mode='a',header=not os.path.exists("pds_header.csv"))
  if is_production_site:
    data_to_be_written_to_csv = pd.DataFrame(columns=["LOCID","PRDID","PRODUCTIONVERSION","ASSEMBLYSCRAP"
    ,"BOMEXPLOSIONRULE","OUTPUTCOEFFICIENT","PDSDEL"
    ,"PDSPRIO","PDSVALIDFR","PDSVALIDTO","PMAXLOTSIZE"
    ,"PMINLOTSIZE","PRODUCTIONVERSIONDESCR","PROUNDING"])
    if (len(input_queues)<3):
      return
    wares=input_queues
    Dict = json.loads(wares)
    wares_amount_data=pd.DataFrame(columns=['wares', 'amount'])
    wares_amount_data['wares'] = Dict.keys()
    
    data_to_be_written_to_csv["PRDID"]=wares_amount_data["wares"]
    #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
    data_to_be_written_to_csv['LOCID']=str(serial)
    data_to_be_written_to_csv["PDSDEL"]=" "
    data_to_be_written_to_csv["PRODUCTIONVERSION"]="01"
    data_to_be_written_to_csv["PDSPRIO"]=1
    data_to_be_written_to_csv["PRODUCTIONVERSIONDESCR"]="come on"
    #IBPproduct_location["VRFKZ"]="X"
    data_to_be_written_to_csv["PDSVALIDFR"]="20160829,172223,X"
    data_to_be_written_to_csv["PDSVALIDTO"]="20240829,172223,X"
    data_to_be_written_to_csv["OUTPUTCOEFFICIENT"]=1
    data_to_be_written_to_csv.to_csv("pds_header.csv",sep=";",index=False,mode='a',header=not os.path.exists("pds_header.csv"))


def read_production_programms_data_from_files(directory):
  dataframes= []
  for root,dirs,files in os.walk(directory):
      for file in files:
         if (file.endswith("productionprograms.csv") & os.path.exists(file)):
            col_names = pd.read_csv(file, nrows=0).columns

            dataframes.append(pd.read_csv(file,sep=";",low_memory=False,header=None,
            names=["serial","name","programm_name","programm_descname","produced_Wares","recruited_workers","consumed_wares_workers","time"]))
  return dataframes


def create_activity_id(x):
  return x["input_wares"]+"+"+x["produced_wares"]+str(x["serial"])

def creat_component_id(x):
  return x["input_wares"]+":"+x["amount"] +"+"+x["produced_wares"]

def write_pds_comp(serial,producedwares,inputwares,programm_descname):
    data_to_be_written_to_csv = pd.DataFrame(columns=["LOCID","PDSCOMPID","PRDID","PRODUCTIONVERSION","ACTID",
    "COMPONENTCOEFFICIENT","COMPVALIDFR","COMPVALIDTO","PDSCOMPDEL","PDSCOMPITEMNR","PRDFR" ])

    if (producedwares != "{}"):
        wares = re.sub(r".*wares\:", "", producedwares)
        Dict = json.loads(wares)
    # wares_amount_data = pd.DataFrame(columns=['produced_wares','amount'])
        wares_amount_data = pd.DataFrame(list(Dict.items()),columns=['produced_wares','amount'])
        #print(wares_amount_data)
        wares_amount_data = wares_amount_data[['produced_wares']]
        #print(wares_amount_data)
    else: return

    if (inputwares != "{}"):
        wares = re.sub(r".*wares\:", "", inputwares)
        Dict = json.loads(wares)
        #wares_amount_input_data = pd.DataFrame(columns= ['input_wares', 'amount'])
        wares_amount_input_data = pd.DataFrame(list(Dict.items()),columns= ['input_wares', 'amount'])
    else: return

    merged_data=wares_amount_data.merge(wares_amount_input_data,how="cross")

    #print(merged_data)
    merged_data["id"]=range(len(merged_data))
    merged_data["serial"]=serial
            
    #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
    data_to_be_written_to_csv["PRDID"]=merged_data["produced_wares"]
    data_to_be_written_to_csv["PRDFR"]=merged_data["input_wares"]
    data_to_be_written_to_csv["PDSCOMPID"]=merged_data.apply(lambda x: creat_component_id(x),axis=1)
    data_to_be_written_to_csv["ACTID"]=merged_data.apply(lambda x: create_activity_id(x),axis=1)
    data_to_be_written_to_csv["PDSCOMPITEMNR"]=merged_data["id"]
    data_to_be_written_to_csv['LOCID']=str(serial)
    data_to_be_written_to_csv["PDSCOMPDEL"]=" "
    data_to_be_written_to_csv["PRODUCTIONVERSION"]="01"
    data_to_be_written_to_csv["COMPVALIDFR"]="20160829,172223,X"
    data_to_be_written_to_csv["COMPVALIDTO"]="20240829,172223,X"
    data_to_be_written_to_csv["COMPONENTCOEFFICIENT"]=1
    data_to_be_written_to_csv.to_csv("pds_comp.csv",sep=";",index=False,mode='a',header=not os.path.exists("pds_comp.csv"))

def write_pds_act(inputrow):
    data_to_be_written_to_csv = pd.DataFrame(columns=["ACTID","LOCID","PRDID","PRODUCTIONVERSION","ACTDESCR","ACTDURATION","ACTSEQNR","PDSACTIVITYDEL"])
    #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
    if inputrow["produced_Wares"]=="{}":
      return
    if inputrow["consumed_wares_workers"] == "{}":
      return
    produced_Wares = json.loads(inputrow["produced_Wares"])
    wares_amount_data = pd.DataFrame(list(produced_Wares.items()),columns= ['produced_Wares', 'amount'])
    PDSCOMPID=[]
    inputwares=json.loads(inputrow["consumed_wares_workers"])
    for i in list(inputwares.keys()  ):
      #print(i)
      PDSCOMPID.append(str(i)+"+"+str(wares_amount_data.iloc[0,0])+str(inputrow["serial"]))
    #print(PDSCOMPID[1])
    data_to_be_written_to_csv["ACTID"]=PDSCOMPID
    data_to_be_written_to_csv['ACTSEQNR']=range(len(PDSCOMPID))
    data_to_be_written_to_csv["PRDID"]=list(produced_Wares.keys())[0]
    #data_to_be_written_to_csv["ACTID"]=programm_descname
    data_to_be_written_to_csv['LOCID']=inputrow["serial"]


    data_to_be_written_to_csv["PDSACTIVITYDEL"]=" "
    data_to_be_written_to_csv["PRODUCTIONVERSION"]="01"

    data_to_be_written_to_csv["ACTDESCR"]=inputrow["programm_descname"]
    
    
    #IBPproduct_location["VRFKZ"]="X"
    data_to_be_written_to_csv.to_csv("pds_act.csv",sep=";",index=False,mode='a',header=not os.path.exists("pds_act.csv"))

def write_product_location_from_production_programs(inputrow):
  
  IBPproduct_location = pd.DataFrame(columns=["LOCID","PRDID","ASSEMBLYSCRAP","DEMANDSUBSTTYPE","DETPLANNINGSCHEDULINGHORIZON",
  "FAIRSHAREEXCESSSTOCKSEGMENT","FAIRSHAREMAXSTOCKSEGMENT","FAIRSHARESAFETYSTOCKSEGMENT","FORECASTCONSPROFILE",
  "FREEZEHORIZONDISTRIBUTION","FREEZEHORIZONPRODUCTION","GOODSRECEIPTPROCTIME","INBOUNDCALID",
  "LOCATIONPRODUCTDEL","MRPCONTROLLER","PLANDELETIONHORIZON","PLANNEDDELIVERYTIME","PMAXLOTSIZE",
  "PMINLOTSIZE","PPINTEGRATIONMODE","PROCUREMENTTYPE","PRODUCTALLOCATIONPROCEDURE","PROUNDING",
  "SAFETYSTOCKMD","SHIPPINGCALID","SOURCEOFSUPPLYSELPROFILE","SUBNETWORKID"])
  if (len(inputrow["produced_Wares"])<3):
    return

  IBPproduct_location["PRDID"]=list(json.loads(inputrow["produced_Wares"]).keys())
  #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
  IBPproduct_location['LOCID']=inputrow["serial"]
  IBPproduct_location["LOCATIONPRODUCTDEL"]=" "
  #IBPproduct_location["VRFKZ"]="X"
  print(IBPproduct_location)
  IBPproduct_location.to_csv("product_location_additionals.csv",sep=";",index=False,mode='a',header=not os.path.exists("product_location_additionals.csv"))

def write_pds_header_additionals(inputrow):

    data_to_be_written_to_csv = pd.DataFrame(columns=["LOCID","PRDID","PRODUCTIONVERSION","ASSEMBLYSCRAP"
    ,"BOMEXPLOSIONRULE","OUTPUTCOEFFICIENT","PDSDEL"
    ,"PDSPRIO","PDSVALIDFR","PDSVALIDTO","PMAXLOTSIZE"
    ,"PMINLOTSIZE","PRODUCTIONVERSIONDESCR","PROUNDING"])


    data_to_be_written_to_csv["PRDID"]=list(json.loads(inputrow["produced_Wares"]).keys())
    #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
    data_to_be_written_to_csv['LOCID']=inputrow["serial"]
    data_to_be_written_to_csv["PDSDEL"]=" "
    data_to_be_written_to_csv["PRODUCTIONVERSION"]="01"
    data_to_be_written_to_csv["PDSPRIO"]=1
    data_to_be_written_to_csv["PRODUCTIONVERSIONDESCR"]="come on"
    data_to_be_written_to_csv["PDSVALIDFR"]="20160829,172223,X"
    data_to_be_written_to_csv["PDSVALIDTO"]="20240829,172223,X"
    data_to_be_written_to_csv["OUTPUTCOEFFICIENT"]=1
    #IBPproduct_location["VRFKZ"]="X"
    data_to_be_written_to_csv.to_csv("pds_header_additionals.csv",sep=";",index=False,mode='a',header=not os.path.exists("pds_header_additionals.csv"))



In [2]:
data = read_data_from_files(directory)


In [3]:
a=map_to_real_time(data)


In [4]:

inputs = a[0].loc[(a[0]["is_production_site"]==1)].query("input_queues.str.len() > 2", engine="python")["input_queues"].apply(split_inputqueus)




In [5]:

write_singel_string_wares_attribute_to_csv_as_IBPproduct(a[3].iloc[0]["wares"])

In [6]:
write_one_buildings_entry_to_IBPstock(a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ].iloc[0],str(a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ].iloc[0]["serial"]))

In [7]:


a[3][["serial","wares","input_queues","is_warehouse","is_production_site"]].drop_duplicates("serial").apply(lambda x: write_product_location(x["serial"],x["wares"],x["input_queues"],x.is_warehouse,x.is_production_site),axis=1)
#a[0].apply(lambda x: test(x["name"],x["serial"]),axis=1)

0         None
363       None
455       None
509       None
666       None
          ... 
349975    None
354089    None
362417    None
396855    None
417274    None
Length: 103, dtype: object

In [8]:
print(a[3])

        serial                  name                    time  \
0          914  amazons_headquarters 2021-01-01 00:33:26.100   
1          914  amazons_headquarters 2021-01-01 01:06:52.300   
2          914  amazons_headquarters 2021-01-01 01:23:35.400   
3          914  amazons_headquarters 2021-01-01 01:40:18.400   
4          914  amazons_headquarters 2021-01-01 02:13:44.600   
...        ...                   ...                     ...   
420330   39548       amazons_furnace 2022-01-01 00:28:01.100   
420331   39548       amazons_furnace 2022-01-01 00:28:21.200   
420332   39548       amazons_furnace 2022-01-01 00:28:21.200   
420333   39548       amazons_furnace 2022-01-01 00:30:01.500   
420334   32218     amazons_warehouse 2022-01-01 00:32:15.900   

        ware_economy_serial  worker_economy_serial position  \
0                         0                      1  (59,53)   
1                         0                      1  (59,53)   
2                         0               

In [9]:


a[3][["serial","wares","input_queues","is_warehouse","is_production_site"]].drop_duplicates("serial").apply(lambda x: write_pds_header(x["serial"],x["wares"],x["input_queues"],x["is_warehouse"],x["is_production_site"]),axis=1)


0         None
363       None
455       None
509       None
666       None
          ... 
349975    None
354089    None
362417    None
396855    None
417274    None
Length: 103, dtype: object

In [10]:
orders=read_orders_data_from_files(directory)
print((orders[0]["time"]))
orders_maped = map_to_real_time(orders)

0          11468
1          11468
2          11468
3          11468
4          11668
          ...   
2430    15706324
2431    15706524
2432    15706724
2433    15706924
2434    15707124
Name: time, Length: 2435, dtype: int64


In [11]:

orders[0].rename(columns={'building_serial':'serial'}, inplace=True)
merged_data = a[0].merge(orders[0], on=["serial","time"], how='left')


In [12]:
print(merged_data[~merged_data["requested_Ware"].isnull()].columns)

Index(['serial', 'name_x', 'time', 'ware_economy_serial',
       'worker_economy_serial', 'position', 'reserved_by_worker', 'workers',
       'is_mine', 'is_port', 'needs_seafaring', 'needs_water_ways', 'passable',
       'is_production_site', 'is_warehouse', 'is_market', 'ware_priorities',
       'wares', 'input_queues', 'produced_Wares', 'workinpositions', 'name_y',
       'requested_Ware', 'amount'],
      dtype='object')


In [13]:
for i in range(len(data)):
  write_series_of_location_to_csv_as_IBPlocation(data[i][["serial","name"]].drop_duplicates(subset="serial"),str(i))




In [14]:
b=data[i][["serial","name"]].drop_duplicates(subset="serial")

In [15]:
write_one_buildings_orders_to_IBPorders(orders[0])

In [16]:
"""
copy= pd.DataFrame(columns=["serial","name","time","wares"])
copy["wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware="log").apply(int).apply(filter_one_or_zero)
#copy.wares= copy.wares.apply(filter_one_or_zero)
copy[["serial","name","time"]] = headquarters[["serial","name","time"]]
#copy.iloc[:,"wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware=i)
sequence =copy.wares.astype(str).str.cat()
#print (sequence)
indices_object = re.finditer(pattern='01', string=sequence)
indices0_1 = [index.start() for index in indices_object]
indices_object = re.finditer(pattern='10', string=sequence)
indices1_0 = [index.start() for index in indices_object]
zeitintervalls =[]
if min(indices1_0) < min(indices0_1):
  indices = sorted(indices1_0.extend(indices0_1))
  if (len(indices)%2 != 0):
    indices.insert(0,0)



copy= pd.DataFrame(columns=["serial","name","time","wares"])
copy["wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware="log")
copy[["serial","name","time"]] = headquarters[["serial","name","time"]]
#copy.iloc[:,"wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware=i)
sequence =copy.wares.astype(str).str.cat()
indices_object = re.finditer(pattern='01', string=sequence)
indices = [index.start() for index in indices_object]

print(indices)
#print(sequence)
#print(sequence.find('6'))




print(indices0_1,indices1_0)


"""

'\ncopy= pd.DataFrame(columns=["serial","name","time","wares"])\ncopy["wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware="log").apply(int).apply(filter_one_or_zero)\n#copy.wares= copy.wares.apply(filter_one_or_zero)\ncopy[["serial","name","time"]] = headquarters[["serial","name","time"]]\n#copy.iloc[:,"wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware=i)\nsequence =copy.wares.astype(str).str.cat()\n#print (sequence)\nindices_object = re.finditer(pattern=\'01\', string=sequence)\nindices0_1 = [index.start() for index in indices_object]\nindices_object = re.finditer(pattern=\'10\', string=sequence)\nindices1_0 = [index.start() for index in indices_object]\nzeitintervalls =[]\nif min(indices1_0) < min(indices0_1):\n  indices = sorted(indices1_0.extend(indices0_1))\n  if (len(indices)%2 != 0):\n    indices.insert(0,0)\n\n\n\ncopy= pd.DataFrame(columns=["serial","name","time","wares"])\ncopy["wares"]=headquarters["wares"].apply(rewrite_wares_t

In [17]:
production_programs=read_production_programms_data_from_files(directory)

In [18]:
production_programs[1].apply(lambda x: write_pds_comp(x["serial"],x["produced_Wares"],x["consumed_wares_workers"],x["programm_descname"]),axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
228    None
229    None
230    None
231    None
232    None
Length: 233, dtype: object

In [19]:
test=production_programs[1].loc[101]

In [20]:
producedwares= test["produced_Wares"]
serial = test["serial"]
programm_descname = test["programm_descname"]
programm_name= test["programm_name"]


In [21]:
def filter_double_products(x):
  if "/" in x["PRDID"].iloc[0]:
    splited = x["PRDID"].iloc[0].rpartition("/")
    #print(splited[2])
    newrow=x
    #print(newrow)
    x.loc["PRDID",0]=splited[0]
    newrow.loc["PRDID",0]=splited[2]
    #print(x.append(newrow,ignore_index = True))
    #print(x)
    #print(newrow)
    return x.concat(newrow,axis=1)
  return 


In [22]:
print(test)

serial                                                                19924
name                                                    amazons_dressmakery
programm_name                                         produce_warriors_coat
programm_descname                                   sewing a warrior’s coat
produced_Wares                                        {"warriors_coat":"1"}
recruited_workers                                                        {}
consumed_wares_workers    {"ironwood":"1","balsa":"2","rubber":"1","gold...
time                                                                3447018
Name: 101, dtype: object


In [23]:
production_programs[1].apply(lambda inputrow:write_pds_act(inputrow),axis=1)
production_programs[1].apply(lambda inputrow:write_product_location_from_production_programs(inputrow),axis=1)
production_programs[1].apply(lambda inputrow:write_pds_header_additionals(inputrow),axis=1)



   LOCID   PRDID ASSEMBLYSCRAP DEMANDSUBSTTYPE DETPLANNINGSCHEDULINGHORIZON  \
0   2351  chisel           NaN             NaN                          NaN   

  FAIRSHAREEXCESSSTOCKSEGMENT FAIRSHAREMAXSTOCKSEGMENT  \
0                         NaN                      NaN   

  FAIRSHARESAFETYSTOCKSEGMENT FORECASTCONSPROFILE FREEZEHORIZONDISTRIBUTION  \
0                         NaN                 NaN                       NaN   

   ... PMAXLOTSIZE PMINLOTSIZE PPINTEGRATIONMODE PROCUREMENTTYPE  \
0  ...         NaN         NaN               NaN             NaN   

  PRODUCTALLOCATIONPROCEDURE PROUNDING SAFETYSTOCKMD SHIPPINGCALID  \
0                        NaN       NaN           NaN           NaN   

  SOURCEOFSUPPLYSELPROFILE SUBNETWORKID  
0                      NaN          NaN  

[1 rows x 27 columns]
   LOCID       PRDID ASSEMBLYSCRAP DEMANDSUBSTTYPE  \
0   2351  felling_ax           NaN             NaN   

  DETPLANNINGSCHEDULINGHORIZON FAIRSHAREEXCESSSTOCKSEGMENT  \
0        

0      None
1      None
2      None
3      None
4      None
       ... 
228    None
229    None
230    None
231    None
232    None
Length: 233, dtype: object